In [7]:
#from matplotlib import pyplot
from typing import List
import plotly.graph_objects as go
import plotly.express as px
#import os

from summer.utils import ref_times_to_dti
from autumn.core.project import get_project, load_timeseries
from autumn.settings.region import Region
from autumn.core.inputs.database import get_input_db

In [8]:
project = get_project("tb_dynamics", Region.KIRIBATI)

In [9]:
update_params = {
    'contact_rate': 0.009981992565289354,
    'infectious_seed': 301.9686161396103,
    'rr_infection_latent': 0.38237582502165823,
    'rr_infection_recovered': 0.7304997270129099,
 }
params = project.param_set.baseline.update(update_params, calibration_format=True)

In [12]:
model = project.run_baseline_model(params)
derived_df_0 = model.get_derived_outputs_df()

In [11]:
input_db = get_input_db()
def find_agebins(x, bins: List[int], labels: List[str]):
    return pd.cut(x, bins, labels=labels, right=False)

pop = input_db.query(table_name="population", conditions={"iso3": "KIR"})

def find_agebins(x, bins: List[int], labels: List[str]):
    return pd.cut(x, bins, labels=labels, right=False)


pop["age_bins"] = find_agebins(
    pop["start_age"], [0, 5, 15, 35, 50, 100], ["0-4", "5-14", "15-34", "35-49", "50+"]
)
pop = pop.groupby(["year", "age_bins"])["population"].agg(["sum"]).reset_index()
pop['sum'] = pop['sum'].astype('int')
pop['type'] = "Empriric"
total_pop = input_db.query(table_name="population", conditions={"iso3": base_params['iso3']})
total_pop["age_bins"] = find_agebins(total_pop["start_age"], [0, 100], ["Total"])
total_pop = total_pop.groupby(["year", "age_bins"])["population"].agg(["sum"]).reset_index()
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(total_pop, x="year", y="sum")
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Actual Population", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

NameError: name 'pd' is not defined